In [ ]:


# Load dataset
file_path = "adult.csv"
data = pd.read_csv(file_path)

# Identify numerical and categorical features
numerical_features = [feature for feature in data.columns if data[feature].dtypes != "O"]
categorical_features = [feature for feature in data.columns if data[feature].dtypes == "O"]

# Apply log transformation to numerical features
log_scaled_numerical_data = data[numerical_features].apply(np.log1p)

# Handle categorical encoding
categorical_encoded_data = data[categorical_features].copy()

# Ordinal encoding for 'education'
education_order = [
    " Preschool", " 1st-4th", " 5th-6th", " 7th-8th", " 9th", " 10th", " 11th", " 12th",
    " Some-college", " Assoc-acdm", " Assoc-voc", " HS-grad", " Bachelors",
    " Masters", " Prof-school", " Doctorate"
]
education_map = {val: idx for idx, val in enumerate(education_order)}
categorical_encoded_data["education"] = categorical_encoded_data["education"].map(education_map)

# Binary encoding for 'sex'
categorical_encoded_data["sex"] = categorical_encoded_data["sex"].map({" Male": 0, " Female": 1})

# One-hot encoding for other categorical variables
one_hot_features = ["workclass", "marital-status", "occupation", "relationship", "race", "native-country"]
categorical_encoded_data = pd.get_dummies(categorical_encoded_data, columns=one_hot_features, drop_first=True)

# Drop NaNs introduced by log transformation
log_scaled_numerical_data = log_scaled_numerical_data.dropna()

# Align categorical data with numerical data index
categorical_encoded_data = categorical_encoded_data.loc[log_scaled_numerical_data.index]

# Concatenate numerical and categorical data
final_data = pd.concat([log_scaled_numerical_data, categorical_encoded_data], axis=1)

# Move target column 'income' to the first column
final_data.insert(0, "income", final_data.pop("income"))

# Convert income to binary (-1 for <=50K, 1 for >50K)
final_data["income"] = final_data["income"].map({" <=50K": -1, " >50K": 1})

# Extract features and target variable
X = final_data.drop(columns=['income']).values.astype(np.float64)  # Ensure numerical values
y = final_data['income'].values.reshape(-1, 1).astype(np.float64)  # Ensure numerical values



Epoch 0: Loss = nan
Epoch 100: Loss = nan


KeyboardInterrupt: 

In [4]:
nan_columns = np.isnan(X).sum(axis=0)  # Count NaNs per feature
print("NaN counts per feature:", nan_columns)
print("Total NaNs in X:", np.isnan(X).sum())


NaN counts per feature: [    0     0     0     0     0     0 22194 22194     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0]
Total NaNs in X: 44388
